<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install gradio langchain elasticsearch transformers sentence_transformers

In [2]:
import gradio as gr
import json
import ast
import requests
import logging
from datetime import datetime
import re
from elasticsearch import Elasticsearch
from transformers import AutoTokenizer, AutoModel
import ast
import logging
import time
import re
import pandas as pd

In [3]:
user_e = "natthaphol.po"
passwords_e = "cnEM5CeFrG"

es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user_e, passwords_e)
)

url = "https://ai-api.manageai.co.th/llm-model-02/generate"
url2 = "https://ai-api.manageai.co.th/llm-model-03/generate"
headers = {'Content-Type': 'application/json'}
username = 'manageai2024'
passwords = 'ManageAI@2024'

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

model_param02 = {
    "max_new_tokens": 70,
    "repetition_penalty": 0.98,
    "stop": ["คำถาม","*","/end"],
}

model_param03 = {
    "max_new_tokens": 120,
    "repetition_penalty": 0.98,
    "stop": ["คำถาม",],
}

model_param = {
    "best_of": 1,
    "frequency_penalty": 1.1,
    "max_new_tokens": 300,
    "repetition_penalty": 1.1,
    "temperature": 0.1,
    "top_k": 10,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "stop": ["assistant"]
}

In [4]:
def generate_response(prompt, model_param, url):
    auth=(username, passwords)
    data = json.dumps({"inputs": prompt, "parameters": model_param})
    response = requests.post(url, data=data, headers=headers ,auth=auth)
    if response.status_code == 200:
        response_json = response.json()
        if "generated_text" in response_json:
            return response_json["generated_text"]
        else:
            print("The 'generated_text' field is missing.")
            return ""
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response content:", response.content)
        return ""

# all

In [32]:
# Predefined patient and policy data
patients = {
    "case 1": {
        "id_card": "0123456789012",
        "title": "นาย",
        "first_name": "สมชาย",
        "last_name": "ใจดี",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "นัดมาตรวจซ้ำ...รับยา",
        "diagnosis": "I10 ความดันโลหิตสูงไม่ทราบสาเหตุ (ปฐมภูมิ) (Essential (primary) hypertension)",
        "billings": "[{'billing_initial': '6301.00', 'billing_discount': '630.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '5671.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '185.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '185.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '800', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '800.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "order_items": "[{'initial': '863', 'item_id': '0102F02B0016O01', 'discount': '86.3', 'item_name': 'Blopress 16 mg', 'net_amount': '776.70', 'item_amount': '10', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '1823', 'item_id': '0102E06I0240O05', 'discount': '182.3', 'item_name': 'Isoptin==SR== ๒๔๐ mg # (กส) ( VERAPAMIL)', 'net_amount': '1640.70', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '750', 'item_id': '0102E04D0030L02', 'discount': '75', 'item_name': 'Dilatam 30 mg (กส)', 'net_amount': '675.00', 'item_amount': '60', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '210', 'item_id': '0102N02A0081L02', 'discount': '21', 'item_name': 'Baby ASA  81  mg ( Enteric coat)', 'net_amount': '189.00', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '2655', 'item_id': '0110E14P0002O01', 'discount': '265.4', 'item_name': 'Livalo 2 mg(กส)', 'net_amount': '2389.60', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '185', 'item_id': 'NSE', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล ผู้ป่วยนอก (Out Patient Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '185', 'item_id': 'NH', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD (OPD Hospital Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '800', 'item_id': 'DF1', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '800.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "total_net": "6841.0"
    },
    "case 2": {
        "id_card": "9876543210987",
        "title": "นางสาว",
        "first_name": "สวยใส",
        "last_name": "ดีใจ",
        "birth_date": "1990-05-15",
        "visit_date": "2024-09-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "หูอื้อสองข้าง 1  วัน",
        "diagnosis": "H699 ความผิดของท่อยูสเตเชียน ไม่ระบุรายละเอียด (Eustachian tube disorder, unspecified)",
        "billings": "[{'billing_initial': '155', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '155.00', 'local_billing_code': '121013', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '125', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '125.00', 'local_billing_code': '140100', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.7(1)', 'billing_net_amount': '100.00', 'local_billing_code': '710301', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'billing_initial': '700', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '700.00', 'local_billing_code': '11', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'billing_initial': '147', 'billing_discount': '13.8', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '133.20', 'local_billing_code': 'PTFEX02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '140', 'billing_discount': '14', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '126.00', 'local_billing_code': 'PTPSE02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '112', 'billing_discount': '11.2', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '100.80', 'local_billing_code': 'PTPRE01', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '155', 'item_id': '121013', 'discount': '0', 'item_name': 'ค่าบริการพยาบาลผู้ป่วยนอก', 'net_amount': '155.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '121013', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '125', 'item_id': '140100', 'discount': '0', 'item_name': 'ค่าบริการทางการแพทย์ผู้ป่วยนอก', 'net_amount': '125.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '140100', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '100', 'item_id': '710301', 'discount': '0', 'item_name': 'การใช้เครื่องมือตรวจ EENT (SMR)', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.7(1)', 'local_billing_code': '710301', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'initial': '700', 'item_id': '11', 'discount': '0', 'item_name': 'ค่าตรวจรักษากรณีผู้ป่วยนอก', 'net_amount': '700.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '11', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'initial': '147', 'item_id': 'PTFEX02', 'discount': '13.8', 'item_name': 'FEXOFENADINE 180 MG (FENAFEX) (A)', 'net_amount': '133.20', 'item_amount': '7', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTFEX02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '140', 'item_id': 'PTPSE02', 'discount': '14', 'item_name': 'PSEUDOEPHEDRINE 60 MG.TABLET  (A)', 'net_amount': '126.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTPSE02', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '112', 'item_id': 'PTPRE01', 'discount': '11.2', 'item_name': 'PREDNISOLONE (FOIL) 5 MG. tablet(B)', 'net_amount': '100.80', 'item_amount': '28', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'PTPRE01', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1440.0",
    },
    "case 3": {
        "id_card": "1111111111111",
        "title": "นางสาว",
        "first_name": "กุ๊กกุ๊ก",
        "last_name": "ไก๊ไก่",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-01",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "ปวดหัว 3 วัน ไม่มีคลื่นไส้",
        "diagnosis": "R510 ปวดศีรษะ ไม่ระบุรายละเอียด (Headache, unspecified (TM))",
        "billings": "[{'billing_initial': '430.00', 'billing_discount': '43.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '387.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '175', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '175.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '175.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '175.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '720', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '720.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "order_items": "[{'initial': '170', 'item_id': '0104B04T0500O06', 'discount': '17', 'item_name': 'TYLENOL 500 MG TAB*ก', 'net_amount': '153.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '260', 'item_id': '0104C17N0250L01', 'discount': '26', 'item_name': 'NAPROXEN 250 MG TAB* [D3]', 'net_amount': '234.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '175', 'item_id': 'NSOP', 'discount': '0', 'item_name': 'บริการพยาบาล OPD', 'net_amount': '175.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '175', 'item_id': 'S-NH0P', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD', 'net_amount': '175.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '0', 'item_id': 'ADS-112', 'discount': '0', 'item_name': 'Adm.score 11-12', 'net_amount': '0.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '720', 'item_id': 'DFO-001', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '720.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}]",
        "total_net": "1457.0",
    },
    "case 4": {
        "id_card": "2222222222222",
        "title": "นางสาว",
        "first_name": "สมศรี",
        "last_name": "มีสุข",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "อุบัติเหตุ",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "\r\nนัดเอ็กซเรย์ก่อนพบแพทย์//ไม่แพ้ยา//อัมพรRN",
        "diagnosis": "S925 กระดูกนิ้วเท้าอื่นหัก (Fracture of other toe)",
        "billings": "[{'billing_initial': '185.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '185.00', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '265.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.2(1)', 'billing_net_amount': '265.00', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'billing_initial': '1060.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.5(1)', 'billing_net_amount': '1060.00', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'billing_initial': '720', 'billing_discount': '0', 'simb_billing_code': '1.2.1(1)', 'billing_net_amount': '720.00', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'billing_initial': '318.00', 'billing_discount': '0.00', 'simb_billing_code': '1.2.1(11)', 'billing_net_amount': '318.00', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.2', 'billing_net_amount': '600.00', 'local_billing_code': '430', 'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}, {'billing_initial': '435', 'billing_discount': '43', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '392.00', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '185', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '185.00', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}]",
        "order_items": "[{'initial': '185', 'item_id': 'NH', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาล OPD (OPD Hospital Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '0', 'item_id': 'APPT', 'discount': '0', 'item_name': 'นัดหมาย (Appointment)', 'net_amount': '0.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '280', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '139', 'item_id': '2029COBA0000001', 'discount': '0', 'item_name': 'ELASTIC BANDAGE 1\" (COBAN 1\") #', 'net_amount': '139.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '68', 'item_id': '2030GAUZ2208005', 'discount': '0', 'item_name': 'GAUZE PAD STERILE 2  X 2  (5 ชิ้น/ห่อ)', 'net_amount': '68.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '58', 'item_id': '2041GLOVSTNP00M', 'discount': '0', 'item_name': 'GLOVE # M ไม่มีแป้ง.(ศรีตรังกล่องเขียว)', 'net_amount': '58.00', 'item_amount': '1', 'simb_billing_code': '1.1.2(1)', 'local_billing_code': '70', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'}, {'initial': '530', 'item_id': 'XL0060', 'discount': '0', 'item_name': 'LT.FOURTH TOE AP', 'net_amount': '530.00', 'item_amount': '1', 'simb_billing_code': '1.1.5(1)', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'initial': '530', 'item_id': 'XL0061', 'discount': '0', 'item_name': 'LT.FOURTH TOE OBLIQUE', 'net_amount': '530.00', 'item_amount': '1', 'simb_billing_code': '1.1.5(1)', 'local_billing_code': '130', 'local_billing_name': 'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์'}, {'initial': '720', 'item_id': 'DF1', 'discount': '0', 'item_name': 'DF แพทย์ตรวจรักษา (Doctor Fee)', 'net_amount': '720.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(1)', 'local_billing_code': '310', 'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'}, {'initial': '159', 'item_id': 'XL0060', 'discount': '0', 'item_name': 'LT.FOURTH TOE AP', 'net_amount': '159.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(11)', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'initial': '159', 'item_id': 'XL0061', 'discount': '0', 'item_name': 'LT.FOURTH TOE OBLIQUE', 'net_amount': '159.00', 'item_amount': '1', 'simb_billing_code': '1.2.1(11)', 'local_billing_code': '410', 'local_billing_name': 'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ '}, {'initial': '600', 'item_id': 'DFO-O-BUDDY', 'discount': '0', 'item_name': 'BUDDY SPINT.', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.2', 'local_billing_code': '430', 'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}, {'initial': '435', 'item_id': '0111D07C1250L07', 'discount': '43', 'item_name': '==  1250 MG.==  CALCIUM CARBONATE', 'net_amount': '392.00', 'item_amount': '30', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '50', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '185', 'item_id': 'NSE', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล ผู้ป่วยนอก (Out Patient Service Charge)', 'net_amount': '185.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '240', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}]",
        "total_net": "3725.0",
    },
    "case 5": {
        "id_card": "3333333333333",
        "title": "นาย",
        "first_name": "สมพร",
        "last_name": "สอนสั่ง",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "ปวดศีรษะ",
        "diagnosis": "K30 อาหารไม่ย่อย (Functional Dyspepsia)",
        "billings": "[{'billing_initial': '80', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '80.00', 'local_billing_code': '1.1.12', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '50', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '50.00', 'local_billing_code': '1.1.14(2)', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '500', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '500.00', 'local_billing_code': '1.2.1', 'local_billing_name': 'Physician Evaluation and Management Services'}, {'billing_initial': '140', 'billing_discount': '14', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '126.00', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '80', 'item_id': 'FIN103', 'discount': '0', 'item_name': '1.1.12  ค่าบริการพยาบาลผู้ป่วยนอก (Nursing Charge OPD)', 'net_amount': '80.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '1.1.12', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '50', 'item_id': 'REGFEE2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู้ป่วยนอก', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '1.1.14(2)', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '500', 'item_id': 'DRFEE', 'discount': '0', 'item_name': 'ตรวจรักษากรณีผู้ป่วยนอก ครั้งแรก (First Out-patient Care)', 'net_amount': '500.00', 'item_amount': '500', 'simb_billing_code': '1.2.1', 'local_billing_code': '1.2.1', 'local_billing_name': 'Physician Evaluation and Management Services'}, {'initial': '140', 'item_id': 'MLOPE', 'discount': '14', 'item_name': 'LOPERAMIDE 2 mg CAP [IMODIUM] [ข] [A]', 'net_amount': '126.00', 'item_amount': '20', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1624.0",
    },
    "case 6": {
        "id_card": "4444444444444",
        "title": "นาย",
        "first_name": "สมบุก",
        "last_name": "สมบัน",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "พบก้อนผิดปกติบริเวณคอหอย",
        "diagnosis": "C140 เนื้องอกร้ายของคอหอย ไม่ระบุรายละเอียด (Malignant neoplasm of other and ill-defined sites in the lip,oral cavity and pharynx, Pharynx unspecified)",
        "billings": "[{'billing_initial': '140', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '140.00', 'local_billing_code': '1.1.12', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '50', 'billing_discount': '0', 'simb_billing_code': '1.1.7(1)', 'billing_net_amount': '50.00', 'local_billing_code': '1.1.7(1)', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'billing_initial': '120', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '120.00', 'local_billing_code': '1.1.14(2)', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '500', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '500.00', 'local_billing_code': '1.2.1', 'local_billing_name': 'Physician Evaluation and Management Services'}, {'billing_initial': '909.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '909.00', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '140', 'item_id': 'REGFEE2', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '140.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': '1.1.12', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '50', 'item_id': 'WV00168', 'discount': '0', 'item_name': 'ไม้กดลิ้น', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.7(1)', 'local_billing_code': '1.1.7(1)', 'local_billing_name': 'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด'}, {'initial': '120', 'item_id': 'REGFEE1', 'discount': '0', 'item_name': 'ค่าบริการทางการแพทย์อื่นๆ', 'net_amount': '120.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': '1.1.14(2)', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '500', 'item_id': 'DRFEE', 'discount': '0', 'item_name': 'ค่าแพทย์', 'net_amount': '500.00', 'item_amount': '500', 'simb_billing_code': '1.2.1', 'local_billing_code': '1.2.1', 'local_billing_name': 'Physician Evaluation and Management Services'}, {'initial': '167', 'item_id': 'E0039', 'discount': '0', 'item_name': 'TRIAMCINOLONE ORAL PASTE ยาทาแผลในปากลดการอักเสบ', 'net_amount': '167.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '742', 'item_id': 'T1263', 'discount': '0', 'item_name': 'CURAM 1 G TAB ยาปฏิชีวนะ', 'net_amount': '742.00', 'item_amount': '14', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1719.0"
    },
    "case 7": {
        "id_card": "6666666666666",
        "title": "นาย",
        "first_name": "ท้อแท้",
        "last_name": "หัวใจ",
        "birth_date": "1966-06-30",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาครั้งแรก",
        "chief_complaint": "ปวดท้อง",
        "diagnosis": "A099 กระเพาะอาหารกับลำไส้อักเสบและลำไส้ใหญ่อักเสบจากสาเหตุที่ไม่ระบุรายละเอียด (Gastroenteritis and colitis of unspecified origin [AGE])",
        "billings": "[{'billing_initial': '1085.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '1085.00', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '180', 'item_id': 'T0757', 'discount': '0', 'item_name': 'GASZYM ยาช่วยย่อยอาหาร', 'net_amount': '180.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '585', 'item_id': 'T1625', 'discount': '0', 'item_name': 'GASMOTIN 5 MG ทำให้ทางเดินอาหารทำงานปกติ[LASA]', 'net_amount': '585.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '320', 'item_id': 'T2031', 'discount': '0', 'item_name': 'VOCINTI 10 mgยับยั้งการหลั่งกรดในกระเพาะอาหาร', 'net_amount': '320.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1220.0",
    }, ##  "diagnosis": "R100 ปวดท้องเฉียบพลัน (Acute abdomen)",
    "case 8": {
        "id_card": "5555555555555",
        "title": "นาย",
        "first_name": "meow",
        "last_name": "meow",
        "birth_date": "1973-01-01",
        "visit_date": "2024-08-20",
        "treatment_type": "เจ็บป่วย",
        "visit_type": "เข้ารับการรักษาต่อเนื่อง",
        "chief_complaint": "พบก้อนผิดปกติบริเวณตับ",
        "diagnosis": "C220 มะเร็งเซลล์ตับ (Liver cell carcinoma)",
        "billings": "[{'billing_initial': '1085.00', 'billing_discount': '0.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '1085.00', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "order_items": "[{'initial': '180', 'item_id': 'T0757', 'discount': '0', 'item_name': 'GASZYM ยาช่วยย่อยอาหาร', 'net_amount': '180.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '585', 'item_id': 'T1625', 'discount': '0', 'item_name': 'GASMOTIN 5 MG ทำให้ทางเดินอาหารทำงานปกติ[LASA]', 'net_amount': '585.00', 'item_amount': '15', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '320', 'item_id': 'T2031', 'discount': '0', 'item_name': 'VOCINTI 10 mgยับยั้งการหลั่งกรดในกระเพาะอาหาร', 'net_amount': '320.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': '1.1.1(3)', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}]",
        "total_net": "1000"
    }
}

policies = {
    "Approve ✅": {
        "effective_date": "2024-01-01",
        "expire_date": "2024-12-31",
        "insurer_name": "บริษัทประกัน A",
        "insurer_contact": "012-345-6789",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "1234567890",
        "holder_cid": "0123456789012",
        "holder_name": "สมชาย ใจดี",
        "service_type": "OPD",
        "claim_date": "2024-08-05",
        "opd_remaining": "10000",
        "general_exclusions": "การทำศัลยกรรมเพื่อความสวยงาม",
        "specific_exclusions": "การรักษาทางทันตกรรมที่ไม่ได้เกิดจากอุบัติเหตุ"
    },
    "uncoverage period 🗓️": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันอุบัติเหตุรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สวยใส ดีใจ",
        "service_type": "OPD",
        "claim_date": "2024-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "opd exceeds 🪙": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันอุบัติเหตุรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "กุ๊กกุ๊ก ไก๊ไก่",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "1000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "uncoverage treatment type 🚑": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมศรี มีสุข",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "โรคร้ายแรง",
        "specific_exclusions": "มะเร็ง เนื้องอก"
    },
    "wait symptoms🩺": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมพร สอนสั่ง",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "10000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "uncoverage exclusion 🚨": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "สมบุก สมบัน",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "100000",
        "general_exclusions": "โรคร้ายแรง",
        "specific_exclusions": "มะเร็ง เนื้องอก"
    },
    "item relate 🩹": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "ท้อแท้ หัวใจ",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "100000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
    "item unrelate 💊": {
        "effective_date": "2025-01-01",
        "expire_date": "2025-12-31",
        "insurer_name": "บริษัทประกัน B",
        "insurer_contact": "098-765-4321",
        "policy_name": "ประกันสุขภาพรายบุคคล",
        "policy_number": "0987654321",
        "holder_cid": "9876543210987",
        "holder_name": "meow meow",
        "service_type": "OPD",
        "claim_date": "2025-09-05",
        "opd_remaining": "100000",
        "general_exclusions": "การบาดเจ็บจากการเล่นกีฬา",
        "specific_exclusions": "การบาดเจ็บจากการปีนเขา"
    },
}

## populate info

In [33]:
def populate_data(selected_patient, selected_policy):
    patient_data = patients.get(selected_patient, {})
    policy_data = policies.get(selected_policy, {})
    return (
        # Patient data
        patient_data.get("id_card", ""),
        f"{patient_data.get('title', '')} {patient_data.get('first_name', '')} {patient_data.get('last_name', '')}",
        patient_data.get("birth_date", ""),
        patient_data.get("visit_date", ""),
        patient_data.get("visit_type", ""),
        patient_data.get("treatment_type", ""),
        patient_data.get("diagnosis", ""),
        patient_data.get("chief_complaint", ""),
        patient_data.get("billings", ""),
        patient_data.get("order_items", ""),
        patient_data.get("total_net", ""),
        # Policy data
        policy_data.get("effective_date", ""),
        policy_data.get("expire_date", ""),
        policy_data.get("insurer_name", ""),
        policy_data.get("insurer_contact", ""),
        policy_data.get("policy_name", ""),
        policy_data.get("policy_number", ""),
        policy_data.get("holder_cid", ""),
        policy_data.get("holder_name", ""),
        policy_data.get("service_type", ""),
        policy_data.get("claim_date", ""),
        policy_data.get("opd_remaining", ""),
        policy_data.get("general_exclusions", ""),
        policy_data.get("specific_exclusions", "")
    )

combined_options = [
    {"label": f"{patient} - {policy}", "patient": patient, "policy": policy}
    for patient, policy in zip(patients.keys(), policies.keys())
]

def populate_data_combined(selected_option):
    # Find the selected patient and policy
    selected_data = next((item for item in combined_options if item["label"] == selected_option), {})
    selected_patient = selected_data.get("patient", "")
    selected_policy = selected_data.get("policy", "")
    return populate_data(selected_patient, selected_policy)

### condition

In [34]:
# Caching class
class InMemoryCache:
    def __init__(self):
        self.cache = {}

    def get(self, key):
        return self.cache.get(key)

    def set(self, key, value):
        self.cache[key] = value

    def clear(self):
        self.cache.clear()

class CachingInferenceClient:
    def __init__(self, base_url, username, password):
        self.base_url = base_url
        self.auth = (username, password)
        self.cache = InMemoryCache()

    def text_generation(self, input_text, params):
        cached_result = self.cache.get(input_text)
        if cached_result:
            logging.info("Cache hit for input text.")
            return cached_result

        try:
            response = requests.post(
                url=f"{self.base_url}/generate",
                data=json.dumps({"inputs": input_text, "parameters": params}),
                headers={'Content-Type': 'application/json'},
                auth=self.auth
            )
            response.raise_for_status()
            result = response.json()
            self.cache.set(input_text, result)
            return result
        except requests.exceptions.RequestException as e:
            logging.error(f"Error during API call: {e}")
            raise

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [35]:
# Function to get drugs from Elasticsearch by ICD-10 code
def icd_elasticsearch(diagnosis):
    icd10_code = diagnosis.split(' ')[0] if diagnosis else None
    search_query = {
        "size": 10000,
        "_source": ["simb_billing_code", "item_name"],
        "query": {"match": {"icd10": icd10_code}}
    }

    # Simulating an Elasticsearch search query
    response = es.search(index="internal-manageai-icd10-order", body=search_query)
    all_drugs = [hit["_source"]["item_name"] for hit in response["hits"]["hits"]]
    return all_drugs

# Function to get missing drugs from the order
def check_missing_drugs(order_items, all_drugs):
    order_drugs = [item['item_name'] for item in order_items]
    missing_drugs = [drug for drug in order_drugs if drug not in all_drugs]
    return missing_drugs

def extract_drug_response_details(drug_response):
    relationship_match = re.search(r'\*\*ยาทั้งหมดมีความสัมพันธ์กับโรคและอาการหลักหรือไม่\?:\*\* \["(.+?)"\]', drug_response)
    relationship_status = relationship_match.group(1) if relationship_match else None
    rationale_match = re.search(r'\*\*คำอธิบาย:\*\* \["(.+?)"\]', drug_response)
    rationale_text = rationale_match.group(1) if rationale_match else None
    return relationship_status, rationale_text

#### kan

In [36]:
def check_drug_relevance_with_llm(diagnosis_output, chief_complaint_output, order_items_output):
    # แปลง order_items_output ให้เป็น DataFrame
    order_items_list = ast.literal_eval(order_items_output)
    order_items_df = pd.DataFrame(order_items_list)

    # แปลง order_items_df ให้เป็น JSON และกรองข้อมูล
    order_items_json = order_items_df.to_dict(orient="records")
    df_test = order_items_df[order_items_df['simb_billing_code'] == '1.1.1(3)']
    own_drugs = ", ".join(df_test['item_name'].values)
    # เรียกใช้ icd_elasticsearch และกรองข้อมูลยา
    all_drugs = icd_elasticsearch(diagnosis_output)
    all_drugs_filtered = [drug for drug in all_drugs if isinstance(drug, str)]
    stock_drugs = ", ".join(all_drugs_filtered)
    print(f"stock_drugs: {stock_drugs}")
    # ตรวจสอบยาที่มีเกิน stock
    all_drug_set = set(all_drugs_filtered)
    own_drugs_list = own_drugs.split(", ")
    missing_drugs = [drug for drug in own_drugs_list if drug not in all_drug_set]
    unusual_drugs = ", ".join(missing_drugs)
    print(f"unusual_drugs: {unusual_drugs}")
    drug_prompt = f"""
YOU ARE A CLINICAL PHARMACOLOGY EXPERT SPECIALIZING IN DRUG INTERACTIONS, MECHANISMS OF ACTION, AND DISEASE PATHOPHYSIOLOGY. YOUR TASK IS TO ASSESS IF THE TEST DRUG AND STOCK DRUGS ARE RELEVANT TO THE GIVEN ICD-10 DIAGNOSIS AND CHIEF COMPLAINT. YOU MUST ANALYZE EACH DRUG'S CLINICAL INDICATIONS, MECHANISM OF ACTION, AND POTENTIAL INTERACTIONS TO DETERMINE RELEVANCE.

### INSTRUCTIONS ###

1. **REVIEW INPUT DATA:**
   - UNDERSTAND the ICD-10 diagnosis and its associated condition or pathophysiology.
   - IDENTIFY how the chief complaint relates to the diagnosis.
   - CHECK the indications and mechanism of action for the test drug.
   - EVALUATE stock drugs to see if they align with the diagnosis and complaint.

2. **EVALUATE RELEVANCE:**
   - COMPARE the test drug and stock drugs with the diagnosis and chief complaint.
   - EXCLUDE any discussion of irrelevant drugs directly; focus only on relevant or partially relevant ones.

3. **CHECK INTERACTIONS:**
   - IDENTIFY any potential drug-drug or drug-disease interactions.
   - FLAG if any drug might worsen the condition or cause adverse effects.

4. **FORMULATE RESPONSE:**
   - STATE whether all drugs are relevant ("Yes" or "No").
   - PROVIDE a clear, concise explanation of your assessment, focusing only on clinically relevant details.
   - *DO NOT include non-relevant drugs or their discussion in the output.*

5. **EDGE CASES:**
   - IF a drug's relevance is unclear, provide a reason and suggest further review.
   - BE CLEAR if no data exists for a drug-disease relationship.

### INPUT FORMAT ### (**Never Response this**)
**ICD-10 Diagnosis:** {diagnosis_output}
**Chief Complaint:** {chief_complaint_output}
**Test Drug:** {unusual_drugs}
**Stock Drug(s):** {unusual_drugs}

### ตัวอย่างคำตอบ ### (**RESPONSE ONLY THIS**)
**ยาทั้งหมดมีความสัมพันธ์กับโรคและอาการหลักหรือไม่?:** ["สัมพันธ์"] หรือ ["ไม่สัมพันธ์"]
**คำอธิบาย:** ["ให้คำอธิบายทางคลินิกโดยละเอียดเกี่ยวกับความเกี่ยวข้อง กลไก และการใช้ยา"]
"""
    drug_response = generate_response(drug_prompt, model_param, url)
    relationship_status, rationale_text = extract_drug_response_details(drug_response)
    print(f"สัมพันธ์: {relationship_status}")
    print(f"เหตุผลการใช้ยา: {rationale_text}")

    return relationship_status, rationale_text


#### gun

In [37]:
claimed_records = set()

# Check for duplicate claims
def duplicate_claim(claim_tuple):
    if claim_tuple in claimed_records:
        return False, "Duplicate claim detected: The claim data has already been submitted."
    claimed_records.add(claim_tuple)
    return True, ""

def uncoverage_exclusions(diagnosis, general_exclusions, specific_exclusions):
    if isinstance(general_exclusions, str):
        general_exclusions = [general_exclusions]
    if isinstance(specific_exclusions, str):
        specific_exclusions = [specific_exclusions]
    all_exclusions = ", ".join(general_exclusions + specific_exclusions)
    prompt_uncoverage_chronic = f"""
คุณเป็นนักวิเคราะห์ประกันภัยที่มีหน้าที่ตรวจสอบว่าการวินิจฉัยทางการแพทย์อยู่ในเงื่อนไขการยกเว้นทั่วไปจากกรมธรรม์หรือไม่ เงื่อนไขการยกเว้นทั่วไปมีดังนี้: {all_exclusions}

กรุณาวิเคราะห์การวินิจฉัยที่ให้มาและตรวจสอบว่าตรงกับหมวดหมู่ใดในเงื่อนไขการยกเว้นข้างต้นหรือไม่ หากตรงกับเงื่อนไขใด ให้ระบุว่า "เข้ากลุ่มประกัน" หากไม่ตรงกับเงื่อนไขใดเลย ให้ระบุว่า "ไม่เข้ากลุ่มใด"

**ลำดับความคิด (Chain of Thoughts):**
1. ระบุคำสำคัญหรือวลีสำคัญในข้อความการวินิจฉัยที่เกี่ยวข้องกับเงื่อนไขการยกเว้น
2. เปรียบเทียบคำสำคัญที่ระบุกับเงื่อนไขการยกเว้นทั่วไปที่ให้มา
3. ตรวจสอบว่าการวินิจฉัยตรงกับหมวดหมู่ใดในเงื่อนไขการยกเว้น
4. สรุปผลลัพธ์เป็น "เข้ากลุ่มประกัน" หรือ "ไม่เข้ากลุ่มใด"
5. ห้ามพูดถึงสิ่งอื่นๆที่ไม่เกี่ยว

**ตัวอย่างการวิเคราะห์:**

**ตัวอย่างที่ 1:**
การวินิจฉัย: "ผู้ป่วยได้รับการวินิจฉัยว่าเป็นมะเร็งปอด"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "เข้ากลุ่มประกัน"
- การวิเคราะห์: การวินิจฉัยระบุว่า "มะเร็งปอด" ซึ่งเป็นโรคที่คุกคามชีวิตและเข้าข่ายในหมวดหมู่ "โรคร้ายแรง"


**ตัวอย่างที่ 2:**
การวินิจฉัย: "ผู้ป่วยเข้ารับการเสริมจมูก"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "เข้ากลุ่มประกัน"
- การวิเคราะห์: การวินิจฉัยระบุว่า "เสริมจมูก" ซึ่งเข้าข่ายหมวดหมู่ "การศัลยกรรมหรือการรักษาความงาม"


**ตัวอย่างที่ 3:**
การวินิจฉัย: "ไข้หวัดทั่วไป"
เงื่อนไขการยกเว้น: ["โรคร้ายแรง", "การศัลยกรรมหรือการรักษาความงาม", "การบาดเจ็บจากกิจกรรมผิดกฎหมาย"]
คำตอบที่ถูกต้อง:
- ผลลัพธ์: "ไม่เข้ากลุ่มใด"
- การวิเคราะห์: การวินิจฉัยระบุว่า "ไข้หวัดทั่วไป" ซึ่งเป็นอาการเจ็บป่วยปกติและไม่คุกคามชีวิต

**งานของคุณ:**

การวินิจฉัย: {diagnosis}
เงื่อนไขการยกเว้น: {all_exclusions}
คำตอบที่ถูกต้อง:
"""
    reasons = ""
    formatted_prompt = prompt_uncoverage_chronic.format(diagnosis=diagnosis, all_exclusions=all_exclusions)
    pattern = r"\b(ไม่เข้ากลุ่มใด|เข้ากลุ่มประกัน)\b"
    cond_output = generate_response(formatted_prompt, model_param=model_param03, url=url)
    match = re.search(pattern, cond_output)

    if match:
        con_re = match.group()
        if con_re  == "ไม่เข้ากลุ่มใด":
            return True, reasons
        elif con_re == "เข้ากลุ่มประกัน":
            reasons  = f"Policy exclusion conditions show uncoverage {all_exclusions}"
            return False, reasons
    else:
        reasons = "cannot Processs"
        return False, reasons

def check_180diseased(icd10_code, diagnosis):
    diagnosis = str(diagnosis)
    icd10_code = diagnosis.split(' ')[0] if diagnosis else None
    symptom = {"K30", "K297", "I495"}
    if icd10_code in symptom:
        result = False
        reason = f"{diagnosis} is a symptom or disease with a waiting period of 180 days"
        return result, reason
    return True, ""

def uncoverage_period(claim_date, effective_date, expire_date):
    try:
        claim_date_obj = datetime.strptime(claim_date, "%Y-%m-%d")
        effective_date_obj = datetime.strptime(effective_date, "%Y-%m-%d")
        expire_date_obj = datetime.strptime(expire_date, "%Y-%m-%d")

        if claim_date_obj < effective_date_obj or claim_date_obj > expire_date_obj:
            return False, f"Claim date {claim_date} is outside the policy coverage period ({effective_date} to {expire_date})."
        return True, ""
    except ValueError:
        return False, "Invalid date format."

def opd_exceeds(opd_remaining, total_net):
    try:
        opd_remaining = float(opd_remaining)
        total_net = float(total_net)
    except ValueError:
        return False, "Invalid number format."
    if total_net > opd_remaining:
        return False, f"Total Billing {total_net} exceeds remaining OPD limit {opd_remaining}."
    return True, ""

def check_and_classify_treatment_type(treatment_type, policy_name):
    # สร้าง prompt สำหรับการจัดกลุ่มประเภทการรักษา
    treatment_type_prompt = f"""
    จัดกลุ่มประเภทการเข้ารับการรักษาตามตัวเลือกต่อไปนี้:
    1. เจ็บป่วย: ปัญหาด้านสุขภาพหรือโรคที่ต้องการการรักษา เช่น ไข้หวัด ไข้หวัดใหญ่ หรืออาการเจ็บป่วยทั่วไป
    2. อุบัติเหตุ: การบาดเจ็บหรือเหตุการณ์ฉุกเฉินที่เกิดจากอุบัติเหตุ เช่น อุบัติเหตุทางรถยนต์ การลื่นล้ม หรือการบาดเจ็บทางกาย
    ประเภทการรักษา: {treatment_type}
    ผลลัพธ์: [สุขภาพ/อุบัติเหตุ]
    หากไม่สามารถจัดกลุ่มได้ ให้ตอบกลับด้วย "ไม่ทราบ"
    หลังจากตอบเสร็จให้ลงท้ายด้วยว่า /end
    """

    vt_output = generate_response(treatment_type_prompt, model_param=model_param02, url=url2)
    if "ไม่ทราบ" in vt_output:
        return False, f"Unable to group treatment type: {treatment_type}. Please check the information."
    pattern = r"\b(สุขภาพ|อุบัติเหตุ)\b"
    match = re.search(pattern, vt_output)
    classified_treatment_type = match.group() if match else "ไม่ทราบ"

    try:
        normalized_treatment_type = classified_treatment_type.strip().lower()
        normalized_policy_name = policy_name.strip().lower()
        health_keywords = ["สุขภาพ", "รายบุคคล", "กลุ่ม"]
        accident_keywords = ["อุบัติเหตุ"]
        if normalized_treatment_type == "สุขภาพ":
            if any(keyword in normalized_policy_name for keyword in health_keywords):
                return True, ""
            else:
                return False, f"Accident insurance cannot be used for medical treatment."
        elif normalized_treatment_type == "อุบัติเหตุ":
            if any(keyword in normalized_policy_name for keyword in accident_keywords):
                return True, ""
            else:
                return False, f"Health insurance cannot be used for accident treatment."
        else:
            return False, f"Treatment type {treatment_type} and insurance policy type {policy_name} do not match."

    except AttributeError:
        return False, "Invalid input types provided"

import ast

def check_drug(order_items, diagnosis):
    if isinstance(order_items, str):
        try:
            order_items = ast.literal_eval(order_items)
        except (ValueError, SyntaxError):
            return "error", "order_items is not a valid JSON or Python-like string."

    if not isinstance(order_items, list):
        return "error", "order_items is not a list."

    for item in order_items:
        if not isinstance(item, dict) or "item_name" not in item:
            return "error", "Each order_item must be a dictionary with 'item_name'."

    stock_drugs = icd_elasticsearch(diagnosis)
    order_drug_names = [item["item_name"] for item in order_items]
    missing_drugs = [drug for drug in order_drug_names if drug not in stock_drugs]
    if missing_drugs:
        result = "Warning ⚠️"
        reason = f"The following drugs are not in stock: {', '.join(missing_drugs)}"
        return result, reason
    return True, ""

## claim validation

In [38]:
def validate_claim(inputs):
    claim_date, effective_date, expire_date = inputs["claim_date"], inputs["effective_date"], inputs["expire_date"]
    opd_remaining, total_net = inputs["opd_remaining"], inputs["total_net"]
    treatment_type, policy_name = inputs["treatment_type"], inputs["policy_name"]
    diagnosis = inputs["diagnosis"]
    general_exclusions = inputs["general_exclusions"]
    specific_exclusions = inputs["specific_exclusions"]
    order_items = inputs["order_items"]

    claim_tuple = (
        inputs["id_card"], inputs["first_name"], inputs["last_name"], inputs["visit_date"], inputs["visit_type"],
        treatment_type, diagnosis, general_exclusions, specific_exclusions, inputs["billing"], order_items, total_net, opd_remaining
    )
    conditions = {
        "period_check": uncoverage_period(claim_date, effective_date, expire_date),
        "opd_check": opd_exceeds(opd_remaining, total_net),
        "treatment_type_check": check_and_classify_treatment_type(treatment_type, policy_name),
        "symptom_check": check_180diseased(None, diagnosis),
        "exclusion_check": uncoverage_exclusions(diagnosis, general_exclusions, specific_exclusions),
        "duplicate_check": duplicate_claim(claim_tuple),
        "drug_check": check_drug(order_items, diagnosis),  # New drug check condition
    }
    status = "Approve ✅"
    reasons = []

    # Update checkboxes and reasons
    for key, (condition_met, reason) in conditions.items():
        if condition_met == "Warning ⚠️":  # Special case for warnings
            status = "Warning ⚠️"
            reasons.append(reason)
            conditions[key] = True  # Tick checkbox
        elif not condition_met:  # Other failed conditions
            status = "Decline ❌"
            reasons.append(reason)
            conditions[key] = True  # Tick checkbox
        else:  # Condition passed
            conditions[key] = False  # Untick checkbox

    return status, "\n\n".join(reasons), conditions


## submit

In [39]:
def on_submit(
    claim_date, effective_date, expire_date, opd_remaining, total_net, treatment_type, policy_name, diagnosis, general_exclusions, specific_exclusions,
    id_card, first_name, last_name, visit_date, visit_type, billing, order_items
):
    inputs = {
        "claim_date": claim_date,
        "effective_date": effective_date,
        "expire_date": expire_date,
        "opd_remaining": opd_remaining,
        "total_net": total_net,
        "treatment_type": treatment_type,
        "policy_name": policy_name,
        "diagnosis": diagnosis,
        "general_exclusions": general_exclusions,
        "specific_exclusions": specific_exclusions,
        "id_card": id_card,
        "first_name": first_name,
        "last_name": last_name,
        "visit_date": visit_date,
        "visit_type": visit_type,
        "billing": billing,
        "order_items": order_items
    }

    status, reasons, conditions = validate_claim(inputs)

    return (
        status,
        reasons,
        conditions["period_check"],
        conditions["opd_check"],
        conditions["treatment_type_check"],
        conditions["symptom_check"],
        conditions["exclusion_check"],
        conditions["duplicate_check"],
        conditions["drug_check"],  # Include drug check in the results
    )


def check_drug_relation(order_items, diagnosis):
    result, reason = check_drug(order_items, diagnosis)
    if result == "Warning ⚠️":
        return gr.update(visible=True), gr.update(value=""), gr.update(visible="")
    else:
        return gr.update(visible=False), gr.update(value=""), gr.update(visible="")

## gradio

In [40]:
# Create Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        select = gr.Dropdown(
            choices=[option["label"] for option in combined_options],
            label="เลือกผู้ป่วยและกรมธรรม์",
            value="",
            interactive=True
        )
    completion_status = gr.Textbox(label="สถานะการทำงาน", interactive=False, visible=False)
    with gr.Row():
        with gr.Column():
            gr.Markdown("### ข้อมูลผู้ป่วย")
            id_card_output = gr.Textbox(label="เลขบัตรประชาชน", interactive = True)
            with gr.Row():
                name_output = gr.Textbox(label="ชื่อ-นามสกุล", interactive = True)
                birth_date_output = gr.Textbox(label="วันเดือนปีเกิด", placeholder="yyyy-mm-dd", interactive = True)
                visit_date_output = gr.Textbox(label="วันที่เข้ารับการรักษา", placeholder="yyyy-mm-dd", interactive = True)
            gr.Markdown("### ข้อมูลการวินิจฉัย")
            with gr.Row():
                treatment_type_output = gr.Dropdown(label="ประเภทการเข้ารักษา", choices=["เจ็บป่วย", "อุบัติเหตุ", "ทันตกรรม"], interactive = True)
                visit_type_output = gr.Dropdown(label="ประเภทการรักษา", choices=["เข้ารับการรักษาครั้งแรก", "เข้ารับการรักษาต่อเนื่อง"], interactive = True)
            with gr.Row():
                chief_complaint_output = gr.Textbox(label="อาการสำคัญ", interactive = True)
                diagnosis_output = gr.Textbox(label="การวินิจฉัย", interactive = True)
            gr.Markdown("### ข้อมูลค่ารักษาพยาบาล")
            with gr.Row():
                billings_output = gr.Textbox(label="รายการค่ารักษาพยาบาล", max_lines = 3, interactive = True)
                order_items_output = gr.Textbox(label="รายการยา/อุปกรณ์การรักษา", max_lines = 3, interactive = True)
            total_net_output = gr.Textbox(label="รวมค่าใช้จ่ายสุทธิ", interactive = True)
        with gr.Column():
            gr.Markdown("### ข้อมูลผู้ถือกรมธรรม์")
            with gr.Row():
                holder_cid_output = gr.Textbox(label="เลขบัตรประชาชนของผู้ถือกรมธรรม์", interactive = True)
                holder_name_output = gr.Textbox(label="ชื่อผู้ถือกรมธรรม์", interactive = True)
                claim_date_output = gr.Textbox(label="วันที่เคลม", interactive = True)
            gr.Markdown("### ข้อมูลประกัน")
            with gr.Row():
                insurer_name_output = gr.Textbox(label="ชื่อบริษัทประกัน", interactive = True)
                insurer_contact_output = gr.Textbox(label="ติดต่อผู้ให้ประกัน", interactive = True)
                policy_number_output = gr.Textbox(label="เลขที่กรมธรรม์", interactive = True)
            with gr.Row():
                policy_name_output = gr.Dropdown(label="แผนประกัน", choices=["ประกันสุขภาพรายบุคคล", "ประกันอุบัติเหตุรายบุคคล", "ประกันสุขภาพกลุ่ม", "ประกันอุบัติเหตุกลุ่ม"], interactive = True)
                service_type_output = gr.Dropdown(label="ประเภทบริการ", choices=["OPD", "IPD"], interactive = True)
            gr.Markdown("### ข้อมูลผลประโยชน์ความคุ้มครอง")
            with gr.Row():
                effective_date_output = gr.Textbox(label="วันที่กรมธรรม์มีผลบังคับ", interactive = True)
                expire_date_output = gr.Textbox(label="วันที่กรมธรรม์สิ้นสุดความคุ้มครอง", interactive = True)
            with gr.Row():
                general_exclusions_output = gr.Textbox(label="ข้อยกเว้นทั่วไป", interactive = True)
                specific_exclusions_output = gr.Textbox(label="ข้อยกเว้นเฉพาะ", interactive = True)
            opd_remaining_output = gr.Textbox(label="วงเงิน OPD คงเหลือ", interactive = True)

    # Bind combined selection to populate data
    select.change(
        populate_data_combined,
        inputs=[select],
        outputs=[
            id_card_output, name_output, birth_date_output, visit_date_output,
            visit_type_output, treatment_type_output, diagnosis_output, chief_complaint_output,
            billings_output, order_items_output, total_net_output, effective_date_output,
            expire_date_output, insurer_name_output, insurer_contact_output, policy_name_output,
            policy_number_output, holder_cid_output, holder_name_output, service_type_output, claim_date_output,
            opd_remaining_output, general_exclusions_output, specific_exclusions_output
        ]
    )
    submit_button = gr.Button("Claim Validate", interactive=True)
    gr.Markdown("### เช็คเงื่อนไขการเคลมแต่ละเงื่อนไข")
    with gr.Row():
        period_check = gr.Checkbox(label="Uncoverage Period", visible=True, interactive=False)
        treatment_type_check = gr.Checkbox(label="Uncoverage Type", visible=True, interactive=False)
        symptom_check = gr.Checkbox(label="Uncoverage Symptom", visible=True, interactive=False)
        exclusion_check = gr.Checkbox(label="Uncoverage Exclusion", visible=True, interactive=False)
        duplicate_check = gr.Checkbox(label="Duplicate Claim", visible=True, interactive=False)
        opd_check = gr.Checkbox(label="OPD Exceeds", visible=True, interactive=False)
        drug_check = gr.Checkbox(label="Check Drug", visible=True, interactive=False)
    with gr.Row():
        coverage_status_output = gr.Textbox(label="สถานะความคุ้มครอง", interactive=False)
        uncoverage_reasons_output = gr.Textbox(label="เหตุผลที่ไม่สามารถเคลมได้", interactive=False)

    submit_button.click(
        on_submit,
        inputs=[
            claim_date_output, effective_date_output, expire_date_output,
            opd_remaining_output, total_net_output, treatment_type_output, policy_name_output,
            diagnosis_output,  general_exclusions_output, specific_exclusions_output, id_card_output, name_output,
            birth_date_output, visit_date_output, visit_type_output, billings_output, order_items_output
        ],
        outputs=[
            coverage_status_output, uncoverage_reasons_output,
            period_check, opd_check, treatment_type_check,
            symptom_check, exclusion_check, duplicate_check, drug_check
        ]
    )
    with gr.Column(visible=False) as drug_check_column:
        check_drug_relate = gr.Button("Check Drug Relation", interactive=True)
        gr.Markdown("### เช็คความเกี่ยวข้องของยากับโรค")
        with gr.Row():
            drug_response = gr.Textbox(label="ผลการวิเคราะห์ความสัมพันธ์ของยากับการวินิจฉัย", interactive=False)
            drug_reason = gr.Textbox(label="เหตุผล", interactive=False)

    submit_button.click(
        check_drug_relation,
        inputs=[order_items_output, diagnosis_output],
        outputs=[drug_check_column, drug_response, drug_reason]
    )

    check_drug_relate.click(
        lambda diagnosis_output, chief_complaint_output, order_items_output: check_drug_relevance_with_llm(
            diagnosis_output, chief_complaint_output, order_items_output
        ),
        inputs=[diagnosis_output, chief_complaint_output, order_items_output],
        outputs=[drug_response, drug_reason]
    )

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b3e9be180e1c96c63f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


stock_drugs: ค่าบริการพยาบาล ผู้ป่วยนอก (Out Pateint Service Charge), ค่าบริการโรงพยาบาล OPD (OPD Hospital Service Charge), DF แพทย์ตรวจรักษา (Doctor Fee)
unusual_drugs: GASZYM ยาช่วยย่อยอาหาร, GASMOTIN 5 MG ทำให้ทางเดินอาหารทำงานปกติ[LASA], VOCINTI 10 mgยับยั้งการหลั่งกรดในกระเพาะอาหาร
สัมพันธ์: ไม่สัมพันธ์
เหตุผลการใช้ยา: GASZYM, GASMOTIN, และ VOCINTI ไม่มีบทบาทในการรักษามะเร็งเซลล์ตับหรือก้อนผิดปกติบริเวณตับ. GASZYM เป็นยาช่วยย่อยอาหาร, GASMOTIN ทำให้ทางเดินอาหารทำงานปกติ, และ VOCINTI ยับยั้งการหลั่งกรดในกระเพาะอาหาร. เหล่านี้ไม่ได้เป็นการรักษาหรือจัดการกับมะเร็งตับ.
stock_drugs: 0.9% NSS INJ. 100 ML, ALUTOP (LOCAL) TAB. 320/75 MG (ก), GASTRO BISMAL 524 MG.TAB.., GASTRO BISMOL 524 MG TAB นอกบ/ช, Gastro Bismal Tab 524 mg. #(กส), MUCOSTA 100 MG TAB, OMEPRAZOLE 20 MG.CAP., MIRACID 20 MG CAP*ก (OMICAP), OMEPRAZOLE 20 MG CAP. (ก), Omeprazole 20 mg (กส), AIR-X TAB., AIR-X [DeGas] 80 MG TAB*ก, Air-x 80 mg, SIMETHICONE TAB. 80 MG (ก), DOMPERIDONE TAB., DOMPERIDONE 10 MG TAB*ก, DOMPERIDONE 1

### debug